In [3]:
import numpy as np
import pandas as pd
import glob
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [91]:
# reading in scores df
scores_files = glob.glob("../original_data/Scores/*.csv")
scores_df = pd.concat((pd.read_csv(f) for f in scores_files))
scores_df.head()

,tourney_year_id,tourney_order,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,loser_seed,match_score_tiebreaks,winner_sets_won,loser_sets_won,winner_games_won,loser_games_won,winner_tiebreaks_won,loser_tiebreaks_won,match_id,match_stats_url_suffix
0,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Finals,1,1,Grigor Dimitrov,d875,grigor-dimitrov,...,3,62 26 63,2,1,14,11,0,0,2017-339-d875-n552,/en/scores/2017/339/MS001/match-stats
1,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Semi-Finals,2,1,Grigor Dimitrov,d875,grigor-dimitrov,...,1,76(7) 62,2,0,13,8,1,0,2017-339-d875-r975,/en/scores/2017/339/MS002/match-stats
2,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Semi-Finals,2,2,Kei Nishikori,n552,kei-nishikori,...,2,76(3) 63,2,0,13,9,1,0,2017-339-n552-w367,/en/scores/2017/339/MS003/match-stats
3,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Quarter-Finals,3,1,Milos Raonic,r975,milos-raonic,...,5,46 63 64,2,1,16,13,0,0,2017-339-r975-n409,/en/scores/2017/339/MS004/match-stats
4,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Quarter-Finals,3,2,Stan Wawrinka,w367,stan-wawrinka,...,NaN,67(2) 64 64,2,1,18,15,0,1,2017-339-w367-e831,/en/scores/2017/339/MS007/match-stats


In [92]:
stats_files = glob.glob("../original_data/Stats/*.csv")
stats_df = pd.concat((pd.read_csv(f) for f in stats_files))
stats_df.head()

,tourney_order,match_id,match_stats_url_suffix,match_time,match_duration,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,...,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_service_games_played,loser_return_games_played,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
0,0,2017-339-r975-n409,/en/scores/2017/339/MS004/match-stats,02:21:00,141.0,23.0,3.0,62.0,97.0,50.0,...,19.0,35.0,1.0,7.0,14.0,15.0,31.0,97.0,84.0,181.0
1,0,2017-339-d875-n552,/en/scores/2017/339/MS001/match-stats,01:49:00,109.0,7.0,2.0,52.0,77.0,41.0,...,13.0,25.0,2.0,7.0,12.0,13.0,24.0,77.0,69.0,146.0
2,0,2017-339-d875-r975,/en/scores/2017/339/MS002/match-stats,01:28:00,88.0,4.0,1.0,36.0,58.0,27.0,...,4.0,22.0,0.0,0.0,10.0,10.0,13.0,58.0,53.0,119.0
3,0,2017-339-n552-w367,/en/scores/2017/339/MS003/match-stats,01:42:00,102.0,1.0,1.0,56.0,77.0,37.0,...,7.0,21.0,1.0,5.0,10.0,11.0,26.0,77.0,63.0,138.0
4,0,2017-339-d875-tb69,/en/scores/2017/339/MS005/match-stats,02:05:00,125.0,3.0,3.0,52.0,94.0,42.0,...,19.0,42.0,1.0,14.0,14.0,14.0,29.0,94.0,82.0,176.0


In [94]:
stats_scores = pd.merge(scores_df, stats_df, on=["match_id", "match_id"])
stats_scores

,tourney_year_id,tourney_order_x,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_service_games_played,loser_return_games_played,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
0,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Finals,1,1,Grigor Dimitrov,d875,grigor-dimitrov,...,13.0,25.0,2.0,7.0,12.0,13.0,24.0,77.0,69.0,146.0
1,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Semi-Finals,2,1,Grigor Dimitrov,d875,grigor-dimitrov,...,4.0,22.0,0.0,0.0,10.0,10.0,13.0,58.0,53.0,119.0
2,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Semi-Finals,2,2,Kei Nishikori,n552,kei-nishikori,...,7.0,21.0,1.0,5.0,10.0,11.0,26.0,77.0,63.0,138.0
3,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Quarter-Finals,3,1,Milos Raonic,r975,milos-raonic,...,19.0,35.0,1.0,7.0,14.0,15.0,31.0,97.0,84.0,181.0
4,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Quarter-Finals,3,2,Stan Wawrinka,w367,stan-wawrinka,...,24.0,52.0,2.0,13.0,16.0,16.0,44.0,119.0,107.0,216.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95768,2016-605,67,nitto-atp-finals,/en/scores/archive/nitto-atp-finals/605/2016/r...,Round Robin,3,8,Dominic Thiem,tb69,dominic-thiem,...,12.0,24.0,3.0,3.0,13.0,13.0,25.0,71.0,74.0,148.0
95769,2016-605,67,nitto-atp-finals,/en/scores/archive/nitto-atp-finals/605/2016/r...,Round Robin,3,9,Andy Murray,mc10,andy-murray,...,11.0,27.0,1.0,5.0,8.0,9.0,19.0,58.0,43.0,103.0
95770,2016-605,67,nitto-atp-finals,/en/scores/archive/nitto-atp-finals/605/2016/r...,Round Robin,3,10,Kei Nishikori,n552,kei-nishikori,...,6.0,23.0,0.0,0.0,9.0,8.0,10.0,43.0,41.0,101.0
95771,2016-605,67,nitto-atp-finals,/en/scores/archive/nitto-atp-finals/605/2016/r...,Round Robin,3,11,Novak Djokovic,d643,novak-djokovic,...,6.0,26.0,0.0,1.0,13.0,13.0,21.0,80.0,75.0,176.0


In [95]:
stats_scores.index = range(stats_scores.shape[0])

In [96]:
stats_scores.head()

,tourney_year_id,tourney_order_x,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_service_games_played,loser_return_games_played,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
0,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Finals,1,1,Grigor Dimitrov,d875,grigor-dimitrov,...,13.0,25.0,2.0,7.0,12.0,13.0,24.0,77.0,69.0,146.0
1,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Semi-Finals,2,1,Grigor Dimitrov,d875,grigor-dimitrov,...,4.0,22.0,0.0,0.0,10.0,10.0,13.0,58.0,53.0,119.0
2,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Semi-Finals,2,2,Kei Nishikori,n552,kei-nishikori,...,7.0,21.0,1.0,5.0,10.0,11.0,26.0,77.0,63.0,138.0
3,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Quarter-Finals,3,1,Milos Raonic,r975,milos-raonic,...,19.0,35.0,1.0,7.0,14.0,15.0,31.0,97.0,84.0,181.0
4,2017-339,1,brisbane,/en/scores/archive/brisbane/339/2017/results,Quarter-Finals,3,2,Stan Wawrinka,w367,stan-wawrinka,...,24.0,52.0,2.0,13.0,16.0,16.0,44.0,119.0,107.0,216.0


In [38]:
file_path = "../original_data/player_rankings/rankings_1973-2017.csv"

rankings_df = pd.read_csv(file_path)
rankings_df.head()

/var/folders/68/x1_csqhx5rlbpjpmwr7hjqw80000gn/T/ipykernel_20411/4007824234.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  rankings_df = pd.read_csv(file_path)


,week_title,week_year,week_month,week_day,rank_text,rank_number,move_positions,move_direction,player_age,ranking_points,tourneys_played,player_url,player_slug,player_id
0,2017.11.20,2017,11,20,1,1,NaN,NaN,31.0,10645,18,/en/players/rafael-nadal/n409/overview,rafael-nadal,n409
1,2017.11.20,2017,11,20,2,2,NaN,NaN,36.0,9605,17,/en/players/roger-federer/f324/overview,roger-federer,f324
2,2017.11.20,2017,11,20,3,3,3.0,up,26.0,5150,23,/en/players/grigor-dimitrov/d875/overview,grigor-dimitrov,d875
3,2017.11.20,2017,11,20,4,4,1.0,down,20.0,4610,25,/en/players/alexander-zverev/z355/overview,alexander-zverev,z355
4,2017.11.20,2017,11,20,5,5,1.0,down,24.0,4015,27,/en/players/dominic-thiem/tb69/overview,dominic-thiem,tb69


In [39]:
rankings = rankings_df.filter(items=['week_title','rank_number','player_id','ranking_points','player_age'])
rankings.head()

,week_title,rank_number,player_id,ranking_points,player_age
0,2017.11.20,1,n409,10645,31.0
1,2017.11.20,2,f324,9605,36.0
2,2017.11.20,3,d875,5150,26.0
3,2017.11.20,4,z355,4610,20.0
4,2017.11.20,5,tb69,4015,24.0


In [40]:
rankings['week_title'] = pd.to_datetime(rankings['week_title'], format = '%Y-%m-%d')
rankings = rankings.rename(columns= {"week_title":"week"})

In [41]:
rankings

,week,rank_number,player_id,ranking_points,player_age
0,2017-11-20,1,n409,10645,31.0
1,2017-11-20,2,f324,9605,36.0
2,2017-11-20,3,d875,5150,26.0
3,2017-11-20,4,z355,4610,20.0
4,2017-11-20,5,tb69,4015,24.0
...,...,...,...,...,...
2694534,1973-08-23,181,h064,0,26.0
2694535,1973-08-23,182,r096,0,23.0
2694536,1973-08-23,183,l083,0,25.0
2694537,1973-08-23,184,s146,0,29.0


In [45]:
scores_df.columns

Index(['tourney_year_id', 'tourney_order', 'tourney_slug',
       'tourney_url_suffix', 'tourney_round_name', 'round_order',
       'match_order', 'winner_name', 'winner_player_id', 'winner_slug',
       'loser_name', 'loser_player_id', 'loser_slug', 'winner_seed',
       'loser_seed', 'match_score_tiebreaks', 'winner_sets_won',
       'loser_sets_won', 'winner_games_won', 'loser_games_won',
       'winner_tiebreaks_won', 'loser_tiebreaks_won', 'match_id',
       'match_stats_url_suffix'],
      dtype='object')

In [47]:
import re

In [52]:
# Reading in player_overview
file_path = "../original_data/player_rankings/player_overviews.csv"

players_df = pd.read_csv(file_path)
players_df.head()

,player_id,player_slug,first_name,last_name,player_url,flag_code,residence,birthplace,birthdate,birth_year,birth_month,birth_day,turned_pro,weight_lbs,weight_kg,height_ft,height_inches,height_cm,handedness,backhand
0,a002,ricardo-acuna,Ricardo,Acuna,http://www.atpworldtour.com/en/players/ricardo...,CHI,"Jupiter, FL, USA","Santiago, Chile",1958.01.13,1958.0,1.0,13.0,0.0,150.0,68.0,"5'9""",69.0,175.0,NaN,NaN
1,a001,sadiq-abdullahi,Sadiq,Abdullahi,http://www.atpworldtour.com/en/players/sadiq-a...,NGR,NaN,NaN,1960.02.02,1960.0,2.0,2.0,0.0,0.0,0.0,"0'0""",0.0,0.0,NaN,NaN
2,a005,nelson-aerts,Nelson,Aerts,http://www.atpworldtour.com/en/players/nelson-...,BRA,NaN,"Cachoeira Do Sul, Brazil",1963.04.25,1963.0,4.0,25.0,0.0,165.0,75.0,"6'2""",74.0,188.0,NaN,NaN
3,a004,egan-adams,Egan,Adams,http://www.atpworldtour.com/en/players/egan-ad...,USA,"Palmetto, FL, USA","Miami Beach, FL, USA",1959.06.15,1959.0,6.0,15.0,0.0,160.0,73.0,"5'10""",70.0,178.0,NaN,NaN
4,a006,ronald-agenor,Ronald,Agenor,http://www.atpworldtour.com/en/players/ronald-...,USA,"Beverly Hills, California, USA","Rabat, Morocco",1964.11.13,1964.0,11.0,13.0,1983.0,180.0,82.0,"5'11""",71.0,180.0,NaN,NaN


In [55]:
players_df['player_name'] = players_df['first_name'] + " " + players_df['last_name']

In [58]:
players_df = players_df.filter(items=['player_id','player_name','flag_code','weight_lbs','height_cm','handedness','backhand'])
players_df.head()

,player_id,player_name,flag_code,weight_lbs,height_cm,handedness,backhand
0,a002,Ricardo Acuna,CHI,150.0,175.0,NaN,NaN
1,a001,Sadiq Abdullahi,NGR,0.0,0.0,NaN,NaN
2,a005,Nelson Aerts,BRA,165.0,188.0,NaN,NaN
3,a004,Egan Adams,USA,160.0,178.0,NaN,NaN
4,a006,Ronald Agenor,USA,180.0,180.0,NaN,NaN


In [139]:
players_df.to_csv('players_test', index=False, header=True)

In [99]:
# get tournament info
tournaments = pd.read_csv("../original_data/Tournaments/tournaments_1877-2017.csv")
tournaments

,tourney_year,tourney_order,tourney_name,tourney_id,tourney_slug,tourney_location,tourney_dates,tourney_month,tourney_day,tourney_singles_draw,...,singles_winner_player_id,doubles_winner_1_name,doubles_winner_1_url,doubles_winner_1_player_slug,doubles_winner_1_player_id,doubles_winner_2_name,doubles_winner_2_url,doubles_winner_2_player_slug,doubles_winner_2_player_id,tourney_year_id
0,1877,1,Wimbledon,540.0,wimbledon,"London, Great Britain",1877.07.09,7.0,9.0,32,...,gi91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1877-540
1,1878,1,Wimbledon,540.0,wimbledon,"London, Great Britain",1878.07.08,7.0,8.0,64,...,hg50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1878-540
2,1879,1,Wimbledon,540.0,wimbledon,"London, Great Britain",1879.07.07,7.0,7.0,64,...,hg35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1879-540
3,1880,1,Wimbledon,540.0,wimbledon,"London, Great Britain",1880.07.05,7.0,5.0,64,...,hg35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1880-540
4,1881,1,Wimbledon,540.0,wimbledon,"London, Great Britain",1881.07.02,7.0,2.0,64,...,rg71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1881-540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4109,2017,63,Intrum Stockholm Open,429.0,stockholm,"Stockholm, Sweden",2017.10.16,10.0,16.0,28,...,d683,Oliver Marach,/en/players/oliver-marach/m760/overview,oliver-marach,m760,Mate Pavic,/en/players/mate-pavic/pf00/overview,mate-pavic,pf00,2017-429
4110,2017,64,Swiss Indoors Basel,328.0,basel,"Basel, Switzerland",2017.10.23,10.0,23.0,32,...,f324,Ivan Dodig,/en/players/ivan-dodig/d646/overview,ivan-dodig,d646,Marcel Granollers,/en/players/marcel-granollers/g710/overview,marcel-granollers,g710,2017-328
4111,2017,65,Erste Bank Open 500,337.0,vienna,"Vienna, Austria",2017.10.23,10.0,23.0,32,...,pf39,Rohan Bopanna,/en/players/rohan-bopanna/b757/overview,rohan-bopanna,b757,Pablo Cuevas,/en/players/pablo-cuevas/c882/overview,pablo-cuevas,c882,2017-337
4112,2017,66,Rolex Paris Masters,352.0,paris,"Paris, France",2017.10.30,10.0,30.0,48,...,sm25,Lukasz Kubot,/en/players/lukasz-kubot/k540/overview,lukasz-kubot,k540,Marcelo Melo,/en/players/marcelo-melo/m919/overview,marcelo-melo,m919,2017-352


In [103]:
# convert tourney date to datetime
tournaments['tourney_dates'] = pd.to_datetime(tournaments['tourney_dates'], format = '%Y-%m-%d')
tournaments

,tourney_year,tourney_order,tourney_name,tourney_id,tourney_slug,tourney_location,tourney_dates,tourney_month,tourney_day,tourney_singles_draw,...,singles_winner_player_id,doubles_winner_1_name,doubles_winner_1_url,doubles_winner_1_player_slug,doubles_winner_1_player_id,doubles_winner_2_name,doubles_winner_2_url,doubles_winner_2_player_slug,doubles_winner_2_player_id,tourney_year_id
0,1877,1,Wimbledon,540.0,wimbledon,"London, Great Britain",1877-07-09,7.0,9.0,32,...,gi91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1877-540
1,1878,1,Wimbledon,540.0,wimbledon,"London, Great Britain",1878-07-08,7.0,8.0,64,...,hg50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1878-540
2,1879,1,Wimbledon,540.0,wimbledon,"London, Great Britain",1879-07-07,7.0,7.0,64,...,hg35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1879-540
3,1880,1,Wimbledon,540.0,wimbledon,"London, Great Britain",1880-07-05,7.0,5.0,64,...,hg35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1880-540
4,1881,1,Wimbledon,540.0,wimbledon,"London, Great Britain",1881-07-02,7.0,2.0,64,...,rg71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1881-540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4109,2017,63,Intrum Stockholm Open,429.0,stockholm,"Stockholm, Sweden",2017-10-16,10.0,16.0,28,...,d683,Oliver Marach,/en/players/oliver-marach/m760/overview,oliver-marach,m760,Mate Pavic,/en/players/mate-pavic/pf00/overview,mate-pavic,pf00,2017-429
4110,2017,64,Swiss Indoors Basel,328.0,basel,"Basel, Switzerland",2017-10-23,10.0,23.0,32,...,f324,Ivan Dodig,/en/players/ivan-dodig/d646/overview,ivan-dodig,d646,Marcel Granollers,/en/players/marcel-granollers/g710/overview,marcel-granollers,g710,2017-328
4111,2017,65,Erste Bank Open 500,337.0,vienna,"Vienna, Austria",2017-10-23,10.0,23.0,32,...,pf39,Rohan Bopanna,/en/players/rohan-bopanna/b757/overview,rohan-bopanna,b757,Pablo Cuevas,/en/players/pablo-cuevas/c882/overview,pablo-cuevas,c882,2017-337
4112,2017,66,Rolex Paris Masters,352.0,paris,"Paris, France",2017-10-30,10.0,30.0,48,...,sm25,Lukasz Kubot,/en/players/lukasz-kubot/k540/overview,lukasz-kubot,k540,Marcelo Melo,/en/players/marcelo-melo/m919/overview,marcelo-melo,m919,2017-352


In [104]:
# tourney columns
tournaments.columns

Index(['tourney_year', 'tourney_order', 'tourney_name', 'tourney_id',
       'tourney_slug', 'tourney_location', 'tourney_dates', 'tourney_month',
       'tourney_day', 'tourney_singles_draw', 'tourney_doubles_draw',
       'tourney_conditions', 'tourney_surface', 'tourney_fin_commit',
       'tourney_url_suffix', 'singles_winner_name', 'singles_winner_url',
       'singles_winner_player_slug', 'singles_winner_player_id',
       'doubles_winner_1_name', 'doubles_winner_1_url',
       'doubles_winner_1_player_slug', 'doubles_winner_1_player_id',
       'doubles_winner_2_name', 'doubles_winner_2_url',
       'doubles_winner_2_player_slug', 'doubles_winner_2_player_id',
       'tourney_year_id'],
      dtype='object')

In [107]:
tourneys = tournaments.filter(items=['tourney_year','tourney_order','tourney_name','tourney_id','tourney_dates'
                                     ,'tourney_singles_draw', 'tourney_surface','tourney_year_id'])
tourneys

,tourney_year,tourney_order,tourney_name,tourney_id,tourney_dates,tourney_singles_draw,tourney_surface,tourney_year_id
0,1877,1,Wimbledon,540.0,1877-07-09,32,Grass,1877-540
1,1878,1,Wimbledon,540.0,1878-07-08,64,Grass,1878-540
2,1879,1,Wimbledon,540.0,1879-07-07,64,Grass,1879-540
3,1880,1,Wimbledon,540.0,1880-07-05,64,Grass,1880-540
4,1881,1,Wimbledon,540.0,1881-07-02,64,Grass,1881-540
...,...,...,...,...,...,...,...,...
4109,2017,63,Intrum Stockholm Open,429.0,2017-10-16,28,Hard,2017-429
4110,2017,64,Swiss Indoors Basel,328.0,2017-10-23,32,Hard,2017-328
4111,2017,65,Erste Bank Open 500,337.0,2017-10-23,32,Hard,2017-337
4112,2017,66,Rolex Paris Masters,352.0,2017-10-30,48,Hard,2017-352


In [108]:
stats_scores2

,tourney_year_id,tourney_order_x,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_service_games_played,loser_return_games_played,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
4060,1991-201,6,maceio,/en/scores/archive/maceio/201/1991/results,Round of 32,5,1,Rodolphe Gilbert,g108,rodolphe-gilbert,...,5.0,14.0,1.0,1.0,9.0,8.0,9.0,38.0,39.0,97.0
4061,1991-201,6,maceio,/en/scores/archive/maceio/201/1991/results,Round of 32,5,2,Danilo Marcelino,m028,danilo-marcelino,...,12.0,20.0,1.0,1.0,9.0,10.0,19.0,58.0,56.0,128.0
4062,1991-201,6,maceio,/en/scores/archive/maceio/201/1991/results,Round of 32,5,3,Pablo Albano,a149,pablo-albano,...,30.0,48.0,7.0,16.0,14.0,14.0,48.0,100.0,102.0,201.0
4063,1991-201,6,maceio,/en/scores/archive/maceio/201/1991/results,Round of 32,5,4,Gabriel Markus,m412,gabriel-markus,...,8.0,24.0,2.0,7.0,9.0,10.0,21.0,64.0,49.0,115.0
4064,1991-201,6,maceio,/en/scores/archive/maceio/201/1991/results,Round of 32,5,5,Jaime Oncins,o076,jaime-oncins,...,3.0,9.0,0.0,0.0,5.0,5.0,4.0,24.0,18.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,2017-891,2,chennai,/en/scores/archive/chennai/891/2017/results,Quarter-Finals,3,3,Benoit Paire,pd31,benoit-paire,...,9.0,23.0,0.0,0.0,7.0,8.0,11.0,45.0,30.0,87.0
45,2017-891,2,chennai,/en/scores/archive/chennai/891/2017/results,Quarter-Finals,3,4,Daniil Medvedev,mm58,daniil-medvedev,...,11.0,18.0,1.0,4.0,9.0,8.0,20.0,55.0,55.0,124.0
40,2017-891,2,chennai,/en/scores/archive/chennai/891/2017/results,Semi-Finals,2,1,Roberto Bautista Agut,bd06,roberto-bautista-agut,...,7.0,17.0,1.0,1.0,9.0,9.0,17.0,53.0,44.0,106.0
41,2017-891,2,chennai,/en/scores/archive/chennai/891/2017/results,Semi-Finals,2,2,Daniil Medvedev,mm58,daniil-medvedev,...,16.0,37.0,3.0,9.0,15.0,15.0,33.0,96.0,90.0,188.0


In [110]:
# Merge tournament informations on matches
stats_scores2 = stats_scores2.merge(tourneys, left_on='tourney_year_id', right_on='tourney_year_id')
stats_scores2

/var/folders/68/x1_csqhx5rlbpjpmwr7hjqw80000gn/T/ipykernel_20411/1539529102.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'tourney_order_x'} in the result is deprecated and will raise a MergeError in a future version.
  stats_scores2 = stats_scores2.merge(tourneys, left_on='tourney_year_id', right_on='tourney_year_id')


,tourney_year_id,tourney_order_x,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,tourney_dates_x,tourney_singles_draw_x,tourney_surface_x,tourney_year_y,tourney_order_y,tourney_name_y,tourney_id_y,tourney_dates_y,tourney_singles_draw_y,tourney_surface_y
0,1991-201,6,maceio,/en/scores/archive/maceio/201/1991/results,Round of 32,5,1,Rodolphe Gilbert,g108,rodolphe-gilbert,...,1991-02-04,32,Hard,1991,6,Guaruja,201.0,1991-02-04,32,Hard
1,1991-201,6,maceio,/en/scores/archive/maceio/201/1991/results,Round of 32,5,2,Danilo Marcelino,m028,danilo-marcelino,...,1991-02-04,32,Hard,1991,6,Guaruja,201.0,1991-02-04,32,Hard
2,1991-201,6,maceio,/en/scores/archive/maceio/201/1991/results,Round of 32,5,3,Pablo Albano,a149,pablo-albano,...,1991-02-04,32,Hard,1991,6,Guaruja,201.0,1991-02-04,32,Hard
3,1991-201,6,maceio,/en/scores/archive/maceio/201/1991/results,Round of 32,5,4,Gabriel Markus,m412,gabriel-markus,...,1991-02-04,32,Hard,1991,6,Guaruja,201.0,1991-02-04,32,Hard
4,1991-201,6,maceio,/en/scores/archive/maceio/201/1991/results,Round of 32,5,5,Jaime Oncins,o076,jaime-oncins,...,1991-02-04,32,Hard,1991,6,Guaruja,201.0,1991-02-04,32,Hard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95768,2017-891,2,chennai,/en/scores/archive/chennai/891/2017/results,Quarter-Finals,3,3,Benoit Paire,pd31,benoit-paire,...,2017-01-02,28,Hard,2017,2,Aircel Chennai Open,891.0,2017-01-02,28,Hard
95769,2017-891,2,chennai,/en/scores/archive/chennai/891/2017/results,Quarter-Finals,3,4,Daniil Medvedev,mm58,daniil-medvedev,...,2017-01-02,28,Hard,2017,2,Aircel Chennai Open,891.0,2017-01-02,28,Hard
95770,2017-891,2,chennai,/en/scores/archive/chennai/891/2017/results,Semi-Finals,2,1,Roberto Bautista Agut,bd06,roberto-bautista-agut,...,2017-01-02,28,Hard,2017,2,Aircel Chennai Open,891.0,2017-01-02,28,Hard
95771,2017-891,2,chennai,/en/scores/archive/chennai/891/2017/results,Semi-Finals,2,2,Daniil Medvedev,mm58,daniil-medvedev,...,2017-01-02,28,Hard,2017,2,Aircel Chennai Open,891.0,2017-01-02,28,Hard


In [121]:
stats_scores2.columns

Index(['tourney_year_id', 'tourney_order_x', 'tourney_slug',
       'tourney_url_suffix', 'tourney_round_name', 'round_order',
       'match_order', 'winner_name', 'winner_player_id', 'winner_slug',
       'loser_name', 'loser_player_id', 'loser_slug', 'winner_seed',
       'loser_seed', 'match_score_tiebreaks', 'winner_sets_won',
       'loser_sets_won', 'winner_games_won', 'loser_games_won',
       'winner_tiebreaks_won', 'loser_tiebreaks_won', 'match_id',
       'match_stats_url_suffix_x', 'tourney_order_y',
       'match_stats_url_suffix_y', 'match_time', 'match_duration',
       'winner_aces', 'winner_double_faults', 'winner_first_serves_in',
       'winner_first_serves_total', 'winner_first_serve_points_won',
       'winner_first_serve_points_total', 'winner_second_serve_points_won',
       'winner_second_serve_points_total', 'winner_break_points_saved',
       'winner_break_points_serve_total', 'winner_service_points_won',
       'winner_service_points_total', 'winner_first_serv

In [128]:
stats_scores3 = stats_scores2.drop(columns=['loser_slug','winner_slug','tourney_url_suffix','tourney_slug','tourney_year_x',
       'tourney_order_x', 'tourney_name_x', 'tourney_id_x', 'tourney_dates_x',
       'tourney_singles_draw_x', 'tourney_surface_x'])

In [129]:
stats_scores3

,tourney_year_id,tourney_round_name,round_order,match_order,winner_name,winner_player_id,loser_name,loser_player_id,winner_seed,loser_seed,...,loser_return_points_total,loser_total_points_won,loser_total_points_total,tourney_year_y,tourney_order_y,tourney_name_y,tourney_id_y,tourney_dates_y,tourney_singles_draw_y,tourney_surface_y
0,1991-201,Round of 32,5,1,Rodolphe Gilbert,g108,Luiz Mattar,m035,NaN,1,...,38.0,39.0,97.0,1991,6,Guaruja,201.0,1991-02-04,32,Hard
1,1991-201,Round of 32,5,2,Danilo Marcelino,m028,Marcelo Filippini,f165,2,NaN,...,58.0,56.0,128.0,1991,6,Guaruja,201.0,1991-02-04,32,Hard
2,1991-201,Round of 32,5,3,Pablo Albano,a149,Horacio De La Pena,d008,Q,3,...,100.0,102.0,201.0,1991,6,Guaruja,201.0,1991-02-04,32,Hard
3,1991-201,Round of 32,5,4,Gabriel Markus,m412,Renzo Furlan,f192,Q,4,...,64.0,49.0,115.0,1991,6,Guaruja,201.0,1991-02-04,32,Hard
4,1991-201,Round of 32,5,5,Jaime Oncins,o076,Mauro Menezes,m059,5,WC,...,24.0,18.0,56.0,1991,6,Guaruja,201.0,1991-02-04,32,Hard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95768,2017-891,Quarter-Finals,3,3,Benoit Paire,pd31,Aljaz Bedene,bh09,5,NaN,...,45.0,30.0,87.0,2017,2,Aircel Chennai Open,891.0,2017-01-02,28,Hard
95769,2017-891,Quarter-Finals,3,4,Daniil Medvedev,mm58,Jozef Kovalik,kc04,NaN,Q,...,55.0,55.0,124.0,2017,2,Aircel Chennai Open,891.0,2017-01-02,28,Hard
95770,2017-891,Semi-Finals,2,1,Roberto Bautista Agut,bd06,Benoit Paire,pd31,2,5,...,53.0,44.0,106.0,2017,2,Aircel Chennai Open,891.0,2017-01-02,28,Hard
95771,2017-891,Semi-Finals,2,2,Daniil Medvedev,mm58,Dudi Sela,sc56,NaN,NaN,...,96.0,90.0,188.0,2017,2,Aircel Chennai Open,891.0,2017-01-02,28,Hard


In [138]:
# arrange matches in chronological order
stats_scores3.sort_values(by = ['tourney_dates_y', 'tourney_round_name', 'round_order','match_order'],
                         ascending = [True, False, False, False])

,tourney_year_id,tourney_round_name,round_order,match_order,winner_name,winner_player_id,loser_name,loser_player_id,winner_seed,loser_seed,...,loser_return_points_total,loser_total_points_won,loser_total_points_total,tourney_year_y,tourney_order_y,tourney_name_y,tourney_id_y,tourney_dates_y,tourney_singles_draw_y,tourney_surface_y
1105,1991-354,Semi-Finals,2,2,Lars Jonsson,j084,Omar Camporese,c009,NaN,6,...,100.0,108.0,230.0,1991,2,Wellington,354.0,1990-12-31,32,Hard
3129,1991-7308,Semi-Finals,2,2,Nicklas Kulti,k181,Magnus Larsson,l206,NaN,NaN,...,65.0,61.0,134.0,1991,1,Adelaide,7308.0,1990-12-31,32,Hard
1104,1991-354,Semi-Finals,2,1,Richard Fromberg,f164,Christian Bergstrom,b040,3,NaN,...,114.0,109.0,233.0,1991,2,Wellington,354.0,1990-12-31,32,Hard
3128,1991-7308,Semi-Finals,2,1,Michael Stich,s351,Jim Courier,c243,6,2,...,80.0,74.0,151.0,1991,1,Adelaide,7308.0,1990-12-31,32,Hard
1091,1991-354,Round of 32,5,16,Markus Zoecke,z045,Daniel Vacek,v141,Q,Q,...,64.0,52.0,114.0,1991,2,Wellington,354.0,1990-12-31,32,Hard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92653,2017-352,1st Round Qualifying,8,5,Laslo Djere,db63,Denis Istomin,i165,NaN,5,...,37.0,40.0,92.0,2017,66,Rolex Paris Masters,352.0,2017-10-30,48,Hard
92652,2017-352,1st Round Qualifying,8,4,Borna Coric,cg80,Benjamin Bonzi,bm95,4,WC,...,58.0,56.0,119.0,2017,66,Rolex Paris Masters,352.0,2017-10-30,48,Hard
92651,2017-352,1st Round Qualifying,8,3,Vasek Pospisil,pd07,Nikoloz Basilashvili,bg23,NaN,3,...,76.0,75.0,157.0,2017,66,Rolex Paris Masters,352.0,2017-10-30,48,Hard
92650,2017-352,1st Round Qualifying,8,2,Jan-Lennard Struff,sl28,Mikhail Kukushkin,k926,2,NaN,...,61.0,63.0,144.0,2017,66,Rolex Paris Masters,352.0,2017-10-30,48,Hard


In [48]:
WL_extensions = ['age', 'entry', 'hand', 'ht', 'player_id', 'flag_code', 'name', 'rank', 'rank_points', 'seed']
player_extensions = ['name','']

def get_features(DF):
    '''
    We replace 'winner' and 'loser' with 'player_1' and 'player_2' (not necessarily in that order)
    'player_1' replaces the name of the player that comes first alphabetically
    The purpose of this is to predict the winner of a match without the data being tied to
    the known winner or loser.
    '''
    DF['player_1'] = pd.concat([DF['winner_name'], DF['loser_name']], axis = 1).min(axis = 1)
    DF['player_2'] = pd.concat([DF['winner_name'], DF['loser_name']], axis = 1).max(axis = 1)
    
    for ext in WL_extensions:
        p1_feature = np.where(DF['player_1'] == DF['winner_name'],
                     DF['winner_' + ext],
                     DF['loser_' + ext])
    
        p2_feature = np.where(DF['player_2'] == DF['winner_name'],
                     DF['winner_' + ext],
                     DF['loser_' + ext])
    
        DF['player_1_' + ext] = p1_feature
        DF['player_2_' + ext] = p2_feature
        
    winner_cols = list(filter(lambda x: x.startswith('winner'), DF.columns))
    loser_cols = list(filter(lambda x: x.startswith('loser'), DF.columns))
    cols_to_drop = winner_cols + loser_cols
    
    target = DF['winner_name']
    
    DF.drop(cols_to_drop, axis = 1, inplace = True)
    
    DF['winner'] = target
    
    return DF

In [51]:
stats_scores.columns

Index(['tourney_year_id', 'tourney_order_x', 'tourney_slug',
       'tourney_url_suffix', 'tourney_round_name', 'round_order',
       'match_order', 'winner_name', 'winner_player_id', 'winner_slug',
       'loser_name', 'loser_player_id', 'loser_slug', 'winner_seed',
       'loser_seed', 'match_score_tiebreaks', 'winner_sets_won',
       'loser_sets_won', 'winner_games_won', 'loser_games_won',
       'winner_tiebreaks_won', 'loser_tiebreaks_won', 'match_id',
       'match_stats_url_suffix_x', 'tourney_order_y',
       'match_stats_url_suffix_y', 'match_time', 'match_duration',
       'winner_aces', 'winner_double_faults', 'winner_first_serves_in',
       'winner_first_serves_total', 'winner_first_serve_points_won',
       'winner_first_serve_points_total', 'winner_second_serve_points_won',
       'winner_second_serve_points_total', 'winner_break_points_saved',
       'winner_break_points_serve_total', 'winner_service_points_won',
       'winner_service_points_total', 'winner_first_serv